In [1]:
import numpy as np
import pandas as pd
from importlib import reload 

import util.performance_metrics as pm
 
reload(pm)

url =  "https://raw.githubusercontent.com/xuda1979/DF/master/sp500_companies/data/constituents.csv"
df_symbols =pd.read_csv('./sp500_companies/data/constituents.csv')

df_symbols.head()
all_symbols = list(df_symbols['Symbol'].unique())
all_symbols.append('SPY')

# Import yfinance
import yfinance as yf  
 
# Get the data for the stock Apple by specifying the stock ticker, start date, and end date
data_all =pd.DataFrame({})
df_ls = []
for symbol in all_symbols:
    
    data = yf.download(symbol)#,'2016-01-01','2018-01-01')
    data['Symbol'] = symbol
    if(len(data)>0):
        df_ls.append(data)
data_all = pd.concat(df_ls)
 

vix=yf.download('^VIX')

symbols = list(data_all['Symbol'].unique())

data_all.reset_index(inplace=True)

data_all.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- NBL: No data found, symbol may be delisted
[*********************100%************

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,1970-01-02,6.851563,6.890625,6.843750,6.851563,1.483355,72000.0,MMM
1,1970-01-05,6.859375,6.898438,6.859375,6.890625,1.491813,446400.0,MMM
2,1970-01-06,6.890625,6.960938,6.882813,6.960938,1.507035,176000.0,MMM
3,1970-01-07,6.960938,7.015625,6.945313,7.000000,1.515492,164800.0,MMM
4,1970-01-08,7.000000,7.109375,6.984375,7.093750,1.535788,304000.0,MMM


In [190]:
vix.reset_index(inplace=True)

In [8]:
import os
paths = os.listdir('./short_data/')

In [11]:
df_ls = []
for path in paths:
    if path[:9] =='NPSXshvol':
        df = pd.read_csv(os.path.join('./short_data', path),sep='|')
        df_ls.append(df)
    
df_short = pd.concat(df_ls)
    

In [20]:
df_short['short_rate'] = df_short['SHORT VOLUME']/df_short['TOTAL VOLUME']

In [21]:
df_short_mp = {}
for symbol in set(df_short.SYMBOL):
    mask = df_short.SYMBOL == symbol
    df = df_short[mask].reset_index(drop=True)
    df_short_mp[symbol] = df


In [48]:
for symbol in df_short_mp:
    df_short_mp[symbol]['DATE']= pd.to_datetime(df_short_mp[symbol]['DATE'], format='%Y%m%d')

In [49]:
df_short_mp['AAPL']

,DATE,SYMBOL,SHORT VOLUME,TOTAL VOLUME,MARKET,short_rate
0,2013-01-25,AAPL,5500,28344,X,0.194045
1,2017-05-01,AAPL,194376,319036,X,0.60926
2,2016-07-12,AAPL,113412,244005,X,0.464794
3,2013-01-31,AAPL,400,3200,X,0.125
4,2012-03-22,AAPL,9620,13420,X,0.716841
...,...,...,...,...,...,...
2584,2018-08-27,AAPL,62080,117933,X,0.526401
2585,2017-05-18,AAPL,87032,191282,X,0.454993
2586,2012-10-31,AAPL,1700,4900,X,0.346939
2587,2015-08-10,AAPL,398843,736133,X,0.541808


# Risk Model

In [931]:
def covariance(df, date, symbols, history):
    df_recent = pd.DataFrame({})
    dates = list(df['Date'].unique())
    dates.sort()
    df_recent['Date']= dates[-(history+1):-1]
    for symbol in symbols:
        mask = df['Symbol'] == symbol
 
        if all([date in list(df[mask]['Date'].unique()) for date in dates[-(history+1):-1]]):
            #print(df[mask]['Close'][-(history+1):-1])
            df_recent[symbol] = df[mask]['Close'].to_list()[-(history+1):-1]
    df_return = pd.DataFrame({})
    df_return['Date'] = df_recent['Date']
    for symbol in df_recent.columns:
        if symbol !='Date':
            #print(symbol,df_recent[symbol].div(df_recent[symbol].shift(1)))
            df_return[symbol] = df_recent[symbol].div(df_recent[symbol].shift(1))
    return df_return[1:].set_index('Date').cov() 
 
            
symbols = ['AAPL', 'MMM']
df_cov = covariance(data_training, list(data_training['Date'].unique())[-1], symbols, 10)
cov=df_cov.to_numpy() 
print(cov)

[[7.85042934e-04 7.44584306e-06]
 [7.44584306e-06 3.88371563e-04]]


In [988]:
def objective(x):
    l = len(x)
    assert l == cov.shape[0]
    return x.dot(cov.dot(x))

def constraint(x):
    return np.atleast_1d(np.sum(np.abs(x)) - 1.5)

 
bounds=[[0, 1], [1, 2]]
res=optimize.minimize(objective, np.array([0, 0]), constraints={"fun": constraint, "type": "ineq"},options={'gtol': 1e-8, 'disp': True}, bounds=bounds )
res.x    

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0006603349124938092
            Iterations: 2
            Function evaluations: 6
            Gradient evaluations: 2


<ipython-input-988-e88c3116b488>:11: OptimizeWarning: Unknown solver options: gtol
  res=optimize.minimize(objective, np.array([0, 0]), constraints={"fun": constraint, "type": "ineq"},options={'gtol': 1e-8, 'disp': True}, bounds=bounds )


array([0.25038093, 1.24961907])

In [1025]:
from scipy.optimize import minimize
def portfolio_optimization(cov, k):        
 
    assert 2*k == cov.shape[0]
    def objective(x):
        return x.dot(cov.dot(x))

    def constraint(x):
        return np.atleast_1d(np.sum(np.abs(x))-1)
    x0 = [1/(2*k)]*k+[-1/(2*k)]*k
    x0 = np.array(x0)
    bounds = []
    for i in range(k):
        bounds.append([1/(4*k),1/k])
    for i in range(k):
        bounds.append([-1/k, -1/(4*k)])
        
    res=optimize.minimize(objective, x0, constraints={"fun": constraint, "type": "ineq"},options={'gtol': 1e-8, 'disp': False}, bounds=bounds )
    s= sum(np.abs(res.x))
    return res.x/s 

# Trade at Open(close-open error as regressor)  

In [81]:
from importlib import reload
import train
import data_processing.data_processing as dp
dp = reload(dp)
train = reload(train)

In [82]:
start_date = '20150101'
end_date = '20190101'
sm_reg_map, sm_reg_map_reverse = train.train_model(data_all, start_date, end_date, symbols,)


In [83]:
sm_reg_map['AAPL'].params

const    0.000142
x1       1.168441
dtype: float64

In [84]:
sm_reg_map_reverse['AAPL'].tvalues

const     0.654053
x1       32.923649
x2       -2.156141
dtype: float64

In [85]:
train.back_testing(
    1,
    data_all,
    symbols,
    sm_reg_map,
    sm_reg_map_reverse,
    '20190101',
    None, 
)

Daily mean return:  0.0037640340695045413
Sharp ratio:  2.8414659581239095
0.7449664429530202


# Model deployment

In [91]:
from importlib import reload
import data_processing.data_processing as dp
import train
train = reload(train)
dp = reload(dp)

In [92]:
 

start_date = '20150101'
end_date = '20191231'
end_date = None
sm_reg_map, sm_reg_map_reverse = train.train_model(data_all, start_date, end_date, symbols)

/usr/local/anaconda3/envs/df/lib/python3.8/site-packages/statsmodels/robust/robust_linear_model.py:287: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  warnings.warn('Estimated scale is 0.0 indicating that the most'


In [93]:
dp.save0(sm_reg_map)
dp.save(sm_reg_map_reverse)

 

In [94]:
df=pd.read_csv('params.csv')

In [95]:
df

,symbol,const,x1,x2,tvalue
0,ECL,0.000076,0.907428,-0.126732,-3.514918
1,FLIR,-0.000019,0.955897,-0.122764,-4.732049
2,CCI,0.000188,0.614708,-0.151185,-3.255271
3,GRMN,0.000075,0.942156,-0.195367,-7.531342
4,PLD,0.000350,0.855984,-0.262958,-5.013329
...,...,...,...,...,...
163,NTAP,0.000459,1.193749,-0.193348,-7.530446
164,OXY,-0.000956,1.036652,-0.068460,-3.249811
165,QRVO,0.000130,1.355739,-0.103351,-3.303907
166,LH,0.000246,0.906864,-0.159404,-4.401495
